In [1]:
import json
from datetime import datetime

In [2]:
current = datetime.now()
current

datetime.datetime(2021, 3, 20, 10, 6, 1, 703696)

In [3]:
current.isoformat()

'2021-03-20T10:06:01.703696'

In [4]:
def format_iso(dt):
    return dt.isoformat()

In [5]:
json.dumps({'a':current})

TypeError: Object of type datetime is not JSON serializable

In [7]:
json.dumps({'a' : current}, default=datetime.isoformat)

'{"a": "2021-03-20T10:06:01.703696"}'

In [9]:
datetime.isoformat(current)

'2021-03-20T10:06:01.703696'

In [10]:
def custom_json_formatter(arg):
    if isinstance(arg, datetime):
        return arg.isoformat()
    if isinstance(arg, set):
        return list(arg)

In [11]:
log = {
    'time' : current,
    'message' : 'Testing...',
    'numbers' : {1, 2, 3}
}

In [12]:
json.dumps(log, default=custom_json_formatter)

'{"time": "2021-03-20T10:06:01.703696", "message": "Testing...", "numbers": [1, 2, 3]}'

In [13]:
log = {
    'time' : current,
    'message' : 'Testing...',
    'numbers' : (1, 2, 3)
}

In [14]:
json.dumps(log, default=custom_json_formatter)

'{"time": "2021-03-20T10:06:01.703696", "message": "Testing...", "numbers": [1, 2, 3]}'

In [15]:
log = {
    'time' : current,
    'message' : 'Testing...',
    'numbers' : (i for i in range(3)) # This is a generator.
}

In [16]:
log

{'time': datetime.datetime(2021, 3, 20, 10, 6, 1, 703696),
 'message': 'Testing...',
 'numbers': <generator object <genexpr> at 0x000001EDBDE46890>}

In [18]:
json.dumps(log, default=custom_json_formatter)

'{"time": "2021-03-20T10:06:01.703696", "message": "Testing...", "numbers": null}'

In [19]:
log['numbers'] = frozenset({1, 2, 3})

In [22]:
json.dumps(log, default=custom_json_formatter)

'{"time": "2021-03-20T10:06:01.703696", "message": "Testing...", "numbers": null}'

This is because our custom_json_formatter function returned None

In [23]:
def isiterable(arg):
    try:
        iter(arg)
    except TypeError:
        return False
    else:
        return True
    
def custom_json_formatter(arg):
    if isiterable(arg):
        return list(arg)
    if isinstance(arg, datetime):
        return arg.isoformat()

In [24]:
isiterable([1,2,3])

True

In [25]:
isiterable((1, 2, 3))

True

In [27]:
isiterable(i for i in range(3))

True

In [28]:
isiterable(frozenset({1, 2, 3}))

True

In [29]:
isiterable({1, 2, 3})

True

In [30]:
isiterable(1)

False

In [31]:
isiterable('Hello')

True

In [32]:
isiterable('a')

True

In [33]:
isiterable(2.4)

False

In [34]:
log = {
    'time' : current,
    'message' : 'Testing...',
    'numbers' : (i for i in range(3))
}

In [35]:
json.dumps(log, default=custom_json_formatter)

'{"time": "2021-03-20T10:06:01.703696", "message": "Testing...", "numbers": [0, 1, 2]}'

In [38]:
log = {
    'time' : current,
    'message' : 'Testing...',
    'numbers' : frozenset(i for i in range(3))
}

In [39]:
json.dumps(log, default=custom_json_formatter)

'{"time": "2021-03-20T10:06:01.703696", "message": "Testing...", "numbers": [0, 1, 2]}'

In [42]:
log = {
    'time' : current,
    'message' : 'Testing...',
    'numbers' : {1, 2, 3}
}

In [43]:
json.dumps(log, default=custom_json_formatter)

'{"time": "2021-03-20T10:06:01.703696", "message": "Testing...", "numbers": [1, 2, 3]}'

In [44]:
def custom_json_formatter(arg):
    if isiterable(arg):
        return list(arg)
    if isinstance(arg, datetime):
        return arg.isoformat()

In [45]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.created = datetime.now()
        
    def __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    def toJson(self):
        return vars(self)

In [46]:
def custom_json_formatter(arg):
    if isinstance(arg, datetime):
        return arg.isoformat()
    if isinstance(arg, Person):
        return arg.toJson()
    if isiterable(arg):
        return list(arg)

In [47]:
p = Person('John', 10)

In [48]:
p

Person(name=John, age=10)

In [49]:
log = {
    'time' : current,
    'message' : 'Testing...',
    'numbers' : {1, 2, 3},
    'person' : p
}

In [50]:
json.dumps(log, default=custom_json_formatter)

'{"time": "2021-03-20T10:06:01.703696", "message": "Testing...", "numbers": [1, 2, 3], "person": {"name": "John", "age": 10, "created": "2021-03-20T10:37:45.047526"}}'

In [51]:
def custom_json_formatter(arg):
    if isinstance(arg, datetime):
        return arg.isoformat()
    elif isiterable(arg):
        return list(arg)
    else:
        try:
            return vars(arg)
        except TypeError:
            return str(arg)

In [53]:
print(json.dumps(log, default=custom_json_formatter, indent=2))

{
  "time": "2021-03-20T10:06:01.703696",
  "message": "Testing...",
  "numbers": [
    1,
    2,
    3
  ],
  "person": {
    "name": "John",
    "age": 10,
    "created": "2021-03-20T10:37:45.047526"
  }
}


In [54]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'Point(x={self.x}, y={self.y})'

In [55]:
pt1 = Point(1, 2)

In [56]:
str(pt1)

'Point(x=1, y=2)'

In [57]:
pt1

Point(x=1, y=2)

In [58]:
from decimal import Decimal

In [59]:
pt2 = Point(Decimal('10.5'), Decimal(1/2))

In [60]:
log['pt1'] = pt1
log['pt2'] = pt2

In [61]:
log

{'time': datetime.datetime(2021, 3, 20, 10, 6, 1, 703696),
 'message': 'Testing...',
 'numbers': {1, 2, 3},
 'person': Person(name=John, age=10),
 'pt1': Point(x=1, y=2),
 'pt2': Point(x=10.5, y=0.5)}

In [62]:
print(json.dumps(log, default=custom_json_formatter, indent=2))

{
  "time": "2021-03-20T10:06:01.703696",
  "message": "Testing...",
  "numbers": [
    1,
    2,
    3
  ],
  "person": {
    "name": "John",
    "age": 10,
    "created": "2021-03-20T10:37:45.047526"
  },
  "pt1": {
    "x": 1,
    "y": 2
  },
  "pt2": {
    "x": "10.5",
    "y": "0.5"
  }
}


In [63]:
from functools import singledispatch

In [72]:
@singledispatch
def format_unserializable(arg):
    try:
        return vars(arg)
    except TypeError:
        return str(arg)

In [73]:
@format_unserializable.register(datetime)
def _(arg):
    return arg.isoformat()

@format_unserializable.register(set)
def _(arg):
    return list(arg)

In [74]:
log

{'time': datetime.datetime(2021, 3, 20, 10, 6, 1, 703696),
 'message': 'Testing...',
 'numbers': <generator object <genexpr> at 0x000001EDBE82B890>,
 'person': Person(name=John, age=10),
 'pt1': Point(x=1, y=2),
 'pt2': Point(x=10.5, y=0.5)}

In [75]:
print(json.dumps(log, default=format_unserializable, indent=2))

{
  "time": "2021-03-20T10:06:01.703696",
  "message": "Testing...",
  "numbers": "<generator object <genexpr> at 0x000001EDBE82B890>",
  "person": {
    "name": "John",
    "age": 10,
    "created": "2021-03-20T10:37:45.047526"
  },
  "pt1": {
    "x": 1,
    "y": 2
  },
  "pt2": {
    "x": "10.5",
    "y": "0.5"
  }
}


In [76]:
log['numbers'] = {1, 2, 3}

In [77]:
log

{'time': datetime.datetime(2021, 3, 20, 10, 6, 1, 703696),
 'message': 'Testing...',
 'numbers': {1, 2, 3},
 'person': Person(name=John, age=10),
 'pt1': Point(x=1, y=2),
 'pt2': Point(x=10.5, y=0.5)}

In [92]:
@singledispatch
def format_unserializable(arg):
    if isiterable(arg):
        return list(arg)
    try:
        return vars(arg)
    except TypeError:
        return str(arg)
    
@format_unserializable.register(datetime)
def _(arg):
    return arg.isoformat()

In [93]:
log['numbers'] = (i for i in range(1, 4))

In [94]:
log

{'time': datetime.datetime(2021, 3, 20, 10, 6, 1, 703696),
 'message': 'Testing...',
 'numbers': <generator object <genexpr> at 0x000001EDBE633740>,
 'person': Person(name=John, age=10),
 'pt1': Point(x=1, y=2),
 'pt2': Point(x=10.5, y=0.5),
 'numbers2': {1, 2, 3}}

In [95]:
print(json.dumps(log, default=format_unserializable, indent=2))

{
  "time": "2021-03-20T10:06:01.703696",
  "message": "Testing...",
  "numbers": [
    1,
    2,
    3
  ],
  "person": {
    "name": "John",
    "age": 10,
    "created": "2021-03-20T10:37:45.047526"
  },
  "pt1": {
    "x": 1,
    "y": 2
  },
  "pt2": {
    "x": "10.5",
    "y": "0.5"
  },
  "numbers2": [
    1,
    2,
    3
  ]
}


In [96]:
@format_unserializable.register(Decimal)
def _(arg):
    return f'Decimal({str(arg)})'

In [97]:
log['numbers2'] = {1, 2, 3}

In [98]:
log

{'time': datetime.datetime(2021, 3, 20, 10, 6, 1, 703696),
 'message': 'Testing...',
 'numbers': <generator object <genexpr> at 0x000001EDBE633740>,
 'person': Person(name=John, age=10),
 'pt1': Point(x=1, y=2),
 'pt2': Point(x=10.5, y=0.5),
 'numbers2': {1, 2, 3}}

In [99]:
print(json.dumps(log, default=format_unserializable, indent=2))

{
  "time": "2021-03-20T10:06:01.703696",
  "message": "Testing...",
  "numbers": [],
  "person": {
    "name": "John",
    "age": 10,
    "created": "2021-03-20T10:37:45.047526"
  },
  "pt1": {
    "x": 1,
    "y": 2
  },
  "pt2": {
    "x": "Decimal(10.5)",
    "y": "Decimal(0.5)"
  },
  "numbers2": [
    1,
    2,
    3
  ]
}


`numbers` in log is empty because it was exhausted